# Object Detection (fish) and Mask Creation (through semantic segmentation)

In this code we use package from (https://github.com/fizyr/keras-maskrcnn) to implement MaskRCNN segmentation/object detection, whihc is proposed by Kaiming He et al. (https://arxiv.org/pdf/1703.06870.pdf), using RetinaNet as base (using Res-Net-152 as a bckbone net).

the purpose of this code is to segment the fish in the images to create a mask. This mask will form the 4th channel to the input array (the 1st 3 channels being the RGB channels of the image) as was suggested in Rathi's et al. (https://ieeexplore.ieee.org/document/8593044), which we implemented in our project and also imrpoved (check the code and the implementation on the Fish4Knowledge dataset).

The code starts by clonning the repositries that contain the Mask-Rcnn packages and pre-trained model on the Google Open Image v4. dataset (https://storage.googleapis.com/openimages/web/factsfigures_v4.html). The pretrained model is trained to detect and segment 300 differetn classes. One of these classes is 'Fish', which we used for our project.

IMPORTANT: The following comment line: # ------------> indicates a directory has to be changed.


## Clonning and Installing The Packages

In [ ]:
!git clone https://github.com/fizyr/keras-maskrcnn.git

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
%cd /content/keras-retinanet/
!pip install .

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
%cd /content/keras-maskrcnn/
!pip install .

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
%matplotlib inline

# reloading modules if they change
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet (the base for the Mask-RCNN)
from keras_maskrcnn import models
from keras_maskrcnn.utils.visualization import draw_mask
from keras_retinanet.utils.visualization import draw_box, draw_caption, draw_annotations
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# other modules/packages
import cv2 as cv
import imutils
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import numpy as np
import time
import glob

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf
tf.__version__

# Setting gpu to 0
setup_gpu(0)

# Getting the type of the GPU being used (Google Coolab)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# ------------> The directory has to be changed according to the directory where the model is stored.
# adjust this to point to your downloaded/trained model
model_path = os.path.join('/content/drive/My Drive', 'resnet152_oid_v1.0.1.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet152')
#print(model.summary())

# load label to names mapping for visualization purposes (Label 249 (Fish) is our target)
labels_to_names = {77: 'Shark', 249: 'Fish', 91:'Gold Fish', 56: 'Sea Turtle', 198: 'Coin'}

## Running Over to Detect, Segment, and Save Fish Images in npy Files



In [ ]:
# List the files (jpg files) in the directory that contain the images in the dataset.
# store the path to these images in the list: List_of_images

# ------------> The directory has to be changed according to the directory where the images dataset folder is stored
list_of_images = glob.glob('/content/drive/My Drive/Nature_Conservancy_dataset/*.jpg')
print(glob.glob('/content/drive/My Drive/Nature_Conservancy_dataset/*.jpg'))
len(list_of_images)

In [ ]:
# Creating the npy files where we will store the images together with their masks after we use
# Mask-RCNN to segment Fish from the images.

# This is the array that will contain the 1306 images 
fish_mask_and_box = np.zeros((1306,500,500,5), dtype = np.uint8) 

# This the array that will contain the dimensions of the original images as well as their number (order in the file)
sizes_and_indices_fish_segmentation  = np.zeros((1306,3), dtype = int)

# This array will contain the class of the fish
fish_classes_fish_segmentation  = np.empty((1306,1), dtype = object)

# This will contain the confidence at which the network detected a fish
confidence_fish_segmentation  = np.zeros((1306,1), dtype = np.float16)

flag = 0
num_of_fish = 0

# iterate over the files to detect and segment the Fish in the dataset images
for j in tqdm(range(len(list_of_images))):

  sizes_and_indices_fish_segmentation[j,0] = int(list_of_images[j].split('/')[-1].split('_')[0])
  fish_classes_fish_segmentation[j,0] = list_of_images[j].split('/')[-1].split('_')[1]

  image = read_image_bgr(list_of_images[j])
  
  # Resizing the images
  if image.shape == (4032, 3024, 3):
    image = imutils.rotate_bound(image, 90)
    print('1')

  elif image.shape == (3024, 3024, 3):
    image = cv.resize(image, (4032, 3024))

  elif image.shape == (3084, 3982, 3):
    image = cv.resize(image, (4032, 3024))
  
  # Storing the images in the npy file (first 3 channels are RGB)
  fish_mask_and_box[j,:,:,0:3] = cv.resize(image, (500,500))
  # print('Image Size' + str(image.shape))

  # copy to draw on
  draw = image.copy()
  draw = cv.cvtColor(draw, cv.COLOR_BGR2RGB)

  # preprocess the image for the Mask-RCNN network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # The following model is used to detect instances of objects in the image and segment them
  start = time.time()
  outputs = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)
  
  # The outputs contain bounding boxes of the objects, the confidence (scores), the labels, and the masks (to segment).
  boxes  = outputs[-4][0]
  scores = outputs[-3][0]
  labels = outputs[-2][0]
  masks  = outputs[-1][0]

  # correct for image scale
  boxes /= scale

  # Storing the masks and the bounding box as images
  if 249 in labels:   # The fish label is No. 249
    for box, score, label, mask in zip(boxes, scores, labels, masks):
      if label == 249:
        color = (255, 255, 255)
        
        # extracting and storing the bunding box and masks
        b = box.astype(int)
        draw_box(draw, b, color= color)

        box_img = np.zeros((3024, 4032, 1), dtype = np.uint8)
        box_img[b[1]: b[3], b[0]: b[2], :] = 255

        mask = mask[:, :, label]
        draw_mask(draw, b, mask, color= (255, 255, 255))

        mask_img = np.zeros((3024, 4032, 3), dtype = np.uint8)
        draw_mask(mask_img, b, mask, color= (255, 255, 255))
        mask_img[mask_img > 50] = 255
        
        fish_mask_and_box[j, :, :, 3] = cv.resize(mask_img[:,:,0], (500,500))
        fish_mask_and_box[j, :, :, 4] = cv.resize(box_img, (500,500))

        confidence_fish_segmentation[j, 0] = score
        
        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        
        # printing the statistics of the fish detected and segmented
        print('We found a fish in: ' + str(j))
        print('The confidence is: ' + str(score))
        num_of_fish +=1
        print('Total Fish: ' + str(num_of_fish))
        print('Percentage of all Fish: ' + str(np.round(num_of_fish/(j+1), decimals = 3)))

        break
    
# ------------> The directory has to be changed according to the directory where the npy files should be stored
np.save('/content/drive/My Drive/fish_mask_and_box.npy', fish_mask_and_box)
np.save('/content/drive/My Drive/sizes_and_indices_fish_segmentation.npy', sizes_and_indices_fish_segmentation)
np.save('/content/drive/My Drive/fish_classes_fish_segmentation.npy', fish_classes_fish_segmentation)
np.save('/content/drive/My Drive/confidence_fish_segmentation.npy', confidence_fish_segmentation)